# Exercise 09 : Plotly

### Matplotlib and Seaborn are really powerful libraries and you can use them for most of the tasks that you may have related to DataViz. But they do not offer you the functionality of creating interactive charts and animations. And Plotly can help you with that. In this exercise, you will need to create almost the same graph as in the previous exercise but in an animation.

In [1]:
import sqlite3
import pandas as pd
import plotly.graph_objects as go

con = sqlite3.connect('../data/logs.db')
cursor = con.cursor()
cursor.execute("ATTACH DATABASE '../data/checking-logs.sqlite' AS logs")
con.commit()
query = """
SELECT *
FROM checker
WHERE uid LIKE 'user_%'
AND status = 'ready'
AND labname = 'project1';
"""
df = pd.io.sql.read_sql(query, con)
con.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['timestamp'] = df['timestamp'].dt.date
df = df.sort_values(["uid", "timestamp"])

df['numTrials'] = 1 
df['cumulative_commits'] = df.groupby('uid')['numTrials'].cumsum()

users = df['uid'].unique()

fig = go.Figure()

for user in users:
    user_df = df[df['uid'] == user]
    fig.add_trace(
        go.Scatter(
            x=user_df['timestamp'],
            y=user_df['cumulative_commits'],
            mode='lines+markers',
            name=user,
            line=dict(width=1),
            marker=dict(size=4)
        )
    )

frames = []
for timestamp in df['timestamp'].unique():
    frame_data = df[df['timestamp'] <= timestamp]
    frames.append(
        go.Frame(
            data=[
                go.Scatter(
                    x=frame_data[frame_data['uid'] == user]['timestamp'],
                    y=frame_data[frame_data['uid'] == user]['cumulative_commits'],
                    mode='lines+markers',
                    name=user
                )
                for user in users
            ],
            name=str(timestamp)
        )
    )

fig.frames = frames

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
                )
            ],
            showactive=False,
            x=0,
            y=1,
            xanchor="left",
            yanchor="top"
        )
    ]
)

fig.update_layout(
    title="Dynamic of commits per user in project1",
    font=dict(size=15),
    xaxis=dict(range=[df['timestamp'].min(), df['timestamp'].max()]),
    yaxis=dict(range=[0, df['cumulative_commits'].max() + 1]),
)

fig.show()